<a href="https://colab.research.google.com/github/Puja176/Intelligent-Cyber-Sec/blob/master/L4_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [20]:
!pip install syft
!pip install numpy

In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

W0703 00:04:20.987358 139914042976128 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0703 00:04:21.006180 139914042976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [8]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [10]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr =  x.send(bob) # send this data to bob

In [13]:
bob._objects

{21908100639: tensor([1, 2, 3, 4, 5])}

In [14]:
x_ptr

(Wrapper)>[PointerTensor | me:93353422196 -> bob:21908100639]

In [15]:
x.location

AttributeError: ignored

In [0]:
x.id_at_location

In [0]:
x.id

In [0]:
x.owner

In [0]:
hook.local_worker

In [0]:
x = x.get()
x

In [0]:
bob._objects

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice


In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice


In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr

In [0]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 

In [0]:
# try the .get() on the pointer


In [0]:
# try the .get(sum_results=True) on your pointer


# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x # a pointer to a remote tensor located at bob

In [0]:
y # a pointer to another remote tensor located at bob

In [0]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

In [0]:
z = z.get()
z

In [0]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

In [0]:
z = z.get() # yup! same results!
z

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()
z = z.get()
z

In [0]:
z.backward() 

In [0]:
x = x.get()

In [0]:
x

In [0]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [21]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

W0703 00:06:14.346796 139914042976128 hook.py:98] Torch was already hooked... skipping hooking process


<VirtualWorker id:bob #objects:1>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# send the data to bob

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [24]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:83287173773 -> bob:52608485449]

In [25]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()
  weights.data.sub_(weights.grad * lr)
  weights.grad *= 0

  print(loss.get().data)

tensor(0.4827)
tensor(0.3459)
tensor(0.2529)
tensor(0.1893)
tensor(0.1455)
tensor(0.1150)
tensor(0.0935)
tensor(0.0781)
tensor(0.0668)
tensor(0.0584)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [0]:
bob._objects

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

In [0]:
del x  # delete the pointer to the remote object


In [0]:
bob._objects

In [0]:
x.child.garbage_collect_data  # True by default

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

In [0]:
x = "asdf"

In [0]:
bob._objects

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

In [0]:
bob._objects

In [0]:
x = "asdf"

In [0]:
bob._objects  # some error from Jupyter is casuing this to appear!

In [0]:
del x

In [0]:
bob._objects

In [0]:
bob = bob.clear_objects() # erase force 
bob._objects

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects # notice that there is only a single tensor in bob

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x.get() + y

In [0]:
z

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x.get() + y.get()
z

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [30]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(7.7476)
tensor(1.4920)
tensor(0.7247)
tensor(0.4702)
tensor(0.3195)
tensor(0.2197)
tensor(0.1523)
tensor(0.1065)
tensor(0.0750)
tensor(0.0532)


In [31]:
# nothing was federated up to this point!

import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")



W0703 00:06:42.392301 139914042976128 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [33]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:99627349447 -> bob:75087141689],
  (Wrapper)>[PointerTensor | me:90852369828 -> bob:95881207475]),
 ((Wrapper)>[PointerTensor | me:88408867086 -> alice:97927151211],
  (Wrapper)>[PointerTensor | me:26995277218 -> alice:7591155773])]

In [0]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [35]:
train()

tensor(0.3330, requires_grad=True)
tensor(1.3671, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [36]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [39]:
bob._objects

{}

In [40]:
alice._objects

{}

In [41]:
y = x + x

KeyError: ignored

In [0]:
y

In [42]:
bob._objects

{}

In [43]:
alice._objects

{}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [45]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [47]:
z = x + y

TensorsNotCollocatedException: ignored

In [48]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:79390936384 -> bob:7718347523]

In [49]:
bob._objects

{}

In [50]:
alice._objects

{}

In [0]:
x = x.get()
x

In [0]:
bob._objects

# Lesson: Pointer Chain Operations

In [51]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{}

In [54]:
alice._objects

{}

In [55]:
x.move(alice)

KeyError: ignored

In [0]:
x

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
bob.clear_objects()
alice.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x

In [0]:
x.remote_get() # inplace operation

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x.move(bob)

In [0]:
x

In [0]:
bob._objects

In [0]:
alice._objects

**Exercise 1:**

- Create a toy dataset
- Create a toy linear model
- Train the dataset using Federated Learning, but do not share the gradiant with the main server, instead use the move function to send the gradients to one of the participating workers, aggregate the model, then send the final model to the main server (i.e., local worker).



---


**Exercise 2:**

While exercise 1 illustrates a good example of preseving privacy in FL, we still had to expose the gradients to one of the participating workers. To avoid this limitation, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
# Exercise 1
import syft as sy
import torch as th
from torch import nn, optim

In [0]:
# create workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [58]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

W0703 00:10:19.204438 139914042976128 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:10:19.208231 139914042976128 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:10:19.212141 139914042976128 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:10:19.214741 139914042976128 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:10:19.216016 139914042976128 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 00:10:19.217597 139914042976128 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secureWorker #objects:0>

In [0]:
#A toy dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)



In [0]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = data[2:4].send(alice)                          

In [115]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:75581486342 -> bob:15751341474],
  (Wrapper)>[PointerTensor | me:74981494240 -> bob:43422138079]),
 ((Wrapper)>[PointerTensor | me:50241149787 -> alice:86165849279],
  (Wrapper)>[PointerTensor | me:67806653791 -> alice:79168021938])]

In [0]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [0]:
# Send copies of the linear model to alice and bob
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)


In [0]:
   # aggreegate the models           
  #model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
  #model.bias.set_(((alice_model.bias.data + bob_model.bias.data) / 2).get())

In [0]:
#train the models at alice and bob
def train(iterations):

    model = nn.Linear(2,1)
    bob_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)

    bob_opt = optim.SGD(params=bob_model.parameters(), lr=0.1)
    alice_opt = optim.SGD(params=alice_model.parameters(), lr=0.1)
    
    for i in range(5):

      
            # do training on the Bob's model
            bob_opt.zero_grad()
            bob_pred = bob_model(data_bob)
            bob_loss = ((bob_pred - target_bob)**2).mean()
            bob_loss.backward()
            bob_opt.step()
            #opt[i] = shrink_opt(opt[i])
           

            # get smarter model back
            bob_loss = bob_loss.get().data
            
            print("Bob_loss",bob_loss)

            
            #do training on the Alice's model
            alice_opt.zero_grad()
            alice_pred = alice_model(data_alice)
            alice_loss = ((alice_pred - target_alice)**2).mean()
            alice_loss.backward()
            alice_opt.step()
            #opt[i] = shrink_opt(opt[i])
            
            #get smarter model back
            
            alice_loss = alice_loss.get().data
            
            print("Alice_loss",alice_loss)
            

            
            # move the models to the third workerz
    bob_model.move(secureWorker)
    alice_model.move(secureWorker)
            
            # aggreegate the models
    with th.no_grad():
        model.weight.set_(((alice_model.weight.data + bob_model.weight.data) / 2).get())
        model.bias.set_(((alice_model.bias.data + bob_model.bias.data) / 2).get())

           
    
    
    

In [151]:
train(1)

Bob_loss tensor(1.8971)
Alice_loss tensor(0.4013)
Bob_loss tensor(0.5616)
Alice_loss tensor(0.3097)
Bob_loss tensor(0.1665)
Alice_loss tensor(0.2573)
Bob_loss tensor(0.0496)
Alice_loss tensor(0.2263)
Bob_loss tensor(0.0150)
Alice_loss tensor(0.2073)
